In [2]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [3]:
background = None
accumulated_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [4]:
def calculate_static_background(frame,alpha_value):
    global background #very first reference image
    
    #No reference is there
    if background is None:
        background=frame.copy().astype('float32') #needed for accumulated Weight
        return None
    
    cv2.accumulateWeighted(frame,background,alpha_value)
    

In [5]:
def segment(frame,threshold=25):
    global background
    
    diff=cv2.absdiff(background.astype('uint8'),frame)
    diff=cv2.medianBlur(diff,7)
    ret_threshold,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    #thresholded=cv2.cvtColor(thresholded,cv2.COLOR_BGR2GRAY)
    
    contours,hierarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    if len(contours)==0:
        return None
    
    hand_segment = max(contours, key=cv2.contourArea)
    return (thresholded,hand_segment)

In [6]:
def show_convex_hull(thresholded,hand_segment):
    
    convex_hull=cv2.convexHull(hand_segment)
    return convex_hull

In [7]:
def count_fingers(thresholded,hand_segment,conv_hull):
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    centre=(cX,cY)
    
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    max_distance = distance.max()
    
    radius = int(0.63 * max_distance)
    circumference = (2 * np.pi * radius)
    
    #Get the roi
    circular_roi=np.zeros(thresholded.shape[:2],dtype="uint8")
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    circular_roi=cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    #Get the contours in circular ring ROI
    contours, hierarchy = cv2.findContours(circular_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    boundingRectangleList=[]
    count=0
    for cnt in contours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        not_out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        not_very_big_contour = ((circumference * 0.25) > cnt.shape[0]) #this make sure that only separate fingers have a bounding rectangle
        
        if not_out_of_wrist and not_very_big_contour:
            boundingRectangleList.append((x,y,w,h))
            count+=1
    
    return (top,bottom,left,right,centre,radius,circular_roi,contours,boundingRectangleList,count)

In [8]:
#final working version
cam=cv2.VideoCapture(0)
num_frames=0

while True:
    ret,frame=cam.read()
    
    
    
    # Grab the ROI from the frame
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    # Apply grayscale and blur to ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    
    if num_frames<60:
        calculate_static_background(gray,0.5)
        res1=cv2.convertScaleAbs(background)
        if num_frames <= 59:
            cv2.putText(frame, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
    
    else:
        hand=segment(gray)
        if hand is not None:
            thresholded,hand_segment=hand
            cv2.drawContours(frame,[hand_segment + (roi_right, roi_top)],-1,(0,0,255),5) #need to pass in brackets if your contour is a single contour not a list of all
            
            conv_hull=show_convex_hull(thresholded,hand_segment)
            cv2.drawContours(frame,[conv_hull + (roi_right, roi_top)],-1,(0,0,255),5) #need to pass in brackets
            
            
            (top,bottom,left,right,centre,radius,circular_roi,contours,boundingRectangleList,count)=count_fingers(thresholded,hand_segment,conv_hull)
            cv2.circle(frame,(top[0] + roi_right, top[1] + roi_top),5,(0,255,0),-1)
            cv2.circle(frame,(bottom[0] + roi_right, bottom[1] + roi_top),5,(0,255,0),-1)
            cv2.circle(frame,(left[0] + roi_right, left[1] + roi_top),5,(0,255,0),-1)
            cv2.circle(frame,(right[0] + roi_right, right[1] + roi_top),5,(0,255,0),-1)
            cv2.circle(frame,(centre[0] + roi_right, centre[1] + roi_top),5,(0,255,0),-1)
            cv2.circle(frame,(centre[0] + roi_right, centre[1] + roi_top),radius,(0,255,0),10)
            
            #draw the contours of the hand at circular boundary
            for index,cnt in enumerate(contours):
                cv2.drawContours(frame,[cnt + (roi_right, roi_top)],-1,(255,0,0),3)
            for rect in boundingRectangleList:
                (x,y,w,h)=rect
                cv2.rectangle(frame,(x+roi_right,y+roi_top),(x+roi_right+w,y+roi_top+h),(255,255,255),5)
            
            #display count
            cv2.putText(frame,str(count),(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255))
            cv2.imshow('Circular ROI',circular_roi)
            cv2.imshow('Thresholded',thresholded)
            
    
    cv2.rectangle(frame, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
    if background is not None:
        cv2.imshow('background',res1)
    cv2.imshow('original',frame)
    
    
    
    
    

    num_frames+=1
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break


cam.release()
cv2.destroyAllWindows()
    